In [1]:
# Import des librairies : 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, BaggingRegressor

import json
import os
import joblib
from datetime import datetime
import sys

In [2]:
# Importation de la configuration des chemins : 
sys.path.append(os.path.abspath(os.path.join("..")))
import config

In [3]:
# Chargement du nom du fichier à partir du fichier de métadonnées :
# Chemin vers le fichier metadata.json depuis le dossier initial : 
metadata_path = os.path.join(config.METADATA_DIR, "metadata.json")

# Lecture du fichier metadata :
with open(metadata_path, "r") as f:
    metadata = json.load(f)

# Récupération du chemin du fichier raw tel qu'enregistré dans metadata : 
processed_data_path = metadata.get("processed_data")

# Chargement du fichier CSV :
df = pd.read_csv(processed_data_path)

In [4]:
# Vérifications du dataset : 
display(df.head())
df.info()

,Cn,M (kg),Ewltp (g/km),Ec (cm3),Ep (KW),Erwltp (g/km),Year,Fc,Ft_Diesel,Ft_Essence,...,Mk_PEUGEOT,Mk_PORSCHE,Mk_RENAULT,Mk_SEAT,Mk_SKODA,Mk_SUBARU,Mk_SUZUKI,Mk_TOYOTA,Mk_VOLKSWAGEN,Mk_VOLVO
0,GIULIA,1504,178,1995,148,1.2,2021,7.9,0,1,...,0,0,0,0,0,0,0,0,0,0
1,GIULIA,1504,179,1995,148,1.2,2021,7.9,0,1,...,0,0,0,0,0,0,0,0,0,0
2,GIULIA,1504,180,1995,148,1.2,2021,8.0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,GIULIA,1520,159,1995,148,0.8,2021,7.0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,GIULIA,1520,160,1995,148,0.8,2021,7.0,0,1,...,0,0,0,0,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172985 entries, 0 to 172984
Data columns (total 49 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Cn                  172985 non-null  object 
 1   M (kg)              172985 non-null  int64  
 2   Ewltp (g/km)        172985 non-null  int64  
 3   Ec (cm3)            172985 non-null  int64  
 4   Ep (KW)             172985 non-null  int64  
 5   Erwltp (g/km)       172985 non-null  float64
 6   Year                172985 non-null  int64  
 7   Fc                  172985 non-null  float64
 8   Ft_Diesel           172985 non-null  int64  
 9   Ft_Essence          172985 non-null  int64  
 10  Mk_ALFA ROMEO       172985 non-null  int64  
 11  Mk_ALLIED VEHICLES  172985 non-null  int64  
 12  Mk_ALPINE           172985 non-null  int64  
 13  Mk_AUDI             172985 non-null  int64  
 14  Mk_BENTLEY          172985 non-null  int64  
 15  Mk_BMW              172985 non-nul

In [5]:
# Séparation de X et y :
X = df.drop(['Ewltp (g/km)', 'Cn', 'Year'], axis=1)
y = df['Ewltp (g/km)']

In [6]:
# Split train/test :
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Normalisation :
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Modèle final: RandomForestRegressor
model_final = RandomForestRegressor(bootstrap=False, max_features=0.75, min_samples_leaf=1,
                                    min_samples_split=9, n_estimators=100, random_state=42)
model_final.fit(X_train_scaled, y_train)
results_model_final = model_final.predict(X_test_scaled)

In [ ]:
# Affichage et enregistrement des metrics : 
# Import des fonctions créées dans le fichier metrics.py : 
from src.utils.metrics import compute_metrics, save_metrics

# Calcul et affichage des metrics : 
metrics = compute_metrics(y_test, results_model_final)
print(f"RMSE: {metrics['rmse']:.4f}")
print(f"R²  : {metrics['r2']:.4f}")

# Enregistrement des metrics : 
metrics_file = os.path.join(config.OUTPUTS_DIR, "metrics.json")
save_metrics(metrics, metrics_file)

In [ ]:
# Analyse erreurs
df_results_final = pd.DataFrame({'y_true': y_test, 'y_pred': results_model_final})
df_results_final['error'] = abs(df_results_final['y_true'] - df_results_final['y_pred'])
seuil = 20
outliers = df_results_final[df_results_final['error'] > seuil]
print(outliers.describe())

In [ ]:
# Enregistrement du modèle : 
# Générer un nom de fichier dynamique pour le modèle :
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"RandomForest_{timestamp}.pkl"

# Construction du chemin complet vers le fichier dans le dossier models existant :
model_path = os.path.join(config.MODELS_DIR, model_filename)

# Enregistrement du modèle entraîné : 
joblib.dump(model_final, model_path)

print(f"Modèle sauvegardé à : {model_path}")

In [43]:
# Chemin complet vers le fichier metadata.json : 
metadata_file = os.path.join(config.METADATA_DIR, "metadata.json")

# Charger le contenu existant de metadata.json s'il existe, sinon initialiser un dictionnaire vide : 
if os.path.exists(metadata_file):
    with open(metadata_file, "r") as f:
        metadata = json.load(f)
else:
    metadata = {}

# Mise à jour ou ajout de la clé pour le modèle entraîné : 
metadata["trained_model"] = model_path

# Réécriture du fichier metadata en conservant les autres informations : 
with open(metadata_file, "w") as f:
    json.dump(metadata, f, indent=4)